# "Euphotic zone residence time of Antarctic Bottom Water"

### Code for Analyzing Unbeaching Kernel Activation 

Corresponding to Yinghuan Xie (yinghuan.xie@utas.edu.au)

In [2]:
# Importing the relevant modules. 
import numpy as np
import xarray as xr
import math
import time
import datetime as dt
from datetime import timedelta
import calendar
import os
import re
from glob import glob
#
import netCDF4 as nc
import pandas as pd
import gsw
#
import cartopy.crs as ccrs
from scipy.io import loadmat
#
import matplotlib.pyplot as plt
import matplotlib.colors as col

#
import cosima_cookbook as cc
session =cc.database.create_session()
expt = '01deg_jra55v13_ryf9091'
#
from os import sys
sys.path.append('/home/581/yx9454/PhD_Ch1')
from My_Py_Func import Ch1_defs as c1d

In [2]:
# In each experiment, we define 06-18, release-06, 05-release, 87-05,
# loop_start_point-87 (if availiable) as No 0,1,2,3,4(if availiable)

master_path = '/g/data/jk72/yx9454/runs/parcels/output_after_Aug/'
traj_input = {'MLS_ON':[{'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Following_MLS_ON/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Following_MLS_ON/'},
                        {'out_freq': 0.25,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Following_Loop_MLS_ON/'}],
              'MLS_OFF':[{'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Following_MLS_OFF/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_OFF/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=5day)_Jan-Dec_MLS_OFF/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=5days)_Following_MLS_OFF/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=5days)_Following_Loop_MLS_OFF/'}],
              'MLS_Weak':[{'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Following_MLS_ON_middle_forw/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON_middle_forw/'},
                          {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON_middle_back/'},
                        {'out_freq': 5,
                        'exp_path':'CORE_Curtain_Backwards(out_freq=6hrs)_Following_MLS_ON_middle_back/'},
                        ]
        }

In [3]:
exp_name = 'MLS_ON'
variables = {'id':'trajectory',  'unbeachCount': 'unbeachCount'  , 'time':'time',
             #'T': 'temperature','S': 'salinity',
             #'ML': 'mixedlayershuffle', 'MLD':'mldepth',
             #'rel_id':'rel_id','rel_z':'rel_z','rel_m':'rel_m',
            }

# Save the unbeachCount files
Only run for the first time

In [4]:
# Load traj files 0
out_freq    = traj_input[exp_name][0]['out_freq']
exp_path    = traj_input[exp_name][0]['exp_path']

# Define the input files
files = sorted(glob(master_path+exp_path+'*.nc'))
files = files[1:]
print(files[0])
print(files[-1])

t = time.time()
#var_in_arrs = c1d.load_OP_traj_files_file_by_file(files,variables)
var_in_arrs_0 = c1d.load_OP_traj_files_file_by_file_v2(files,variables)
print(time.time()-t)


/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Following_MLS_ON/Particles_001.nc
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Following_MLS_ON/Particles_158.nc
id
unbeachCount
time
374.0430021286011


In [5]:
# Load traj files 1
out_freq    = traj_input[exp_name][1]['out_freq']
exp_path    = traj_input[exp_name][1]['exp_path']
dirs_1 = sorted(glob(master_path+exp_path+'*'))

# Files 1 are for 2005.01.01-2006.01.01,
# we applied a time selection to make sure no time step beyond is captured.
# Time here is in nanoseconds (1000000000 times of seconds) since 1970,1,1
time_max =  (dt.datetime(2006,1,1)  - dt.datetime(1970,1,1)).total_seconds()*1000000000
time_min =  (dt.datetime(2004,12,31)- dt.datetime(1970,1,1)).total_seconds()*1000000000

list_of_var_in_arrs = []
for i in dirs_1:
    print(i)
    files = sorted(glob(i+'/*.nc'))
    # Note that 'traj' is continous npts from 0, 
    # while 'trajectory' is particle id inherits from previous file
    t = time.time()
    #var_in_arrs = c1d.load_OP_traj_files_file_by_file(files,variables)
    var_in_arrs = c1d.load_OP_traj_files_file_by_file_v2(files,variables)
    print(time.time()-t)
    
    #Select obs within 2005
    obs_mask = (var_in_arrs['time'][0,:]<time_max) & (var_in_arrs['time'][0,:]>time_min)
    for v in variables:
        if v != 'id':
            #print(v)
            var_in_arrs[v] = var_in_arrs[v][:,obs_mask]
            nan_arr = np.zeros((var_in_arrs[v].shape[0],int(365/out_freq)-var_in_arrs[v].shape[1]))/0
            var_in_arrs[v] = np.concatenate((nan_arr,var_in_arrs[v]),axis=1)
            #var_in_arrs_forw_1st_year = 
    list_of_var_in_arrs.append(var_in_arrs)
        
# Combine each monthly array into one array        
var_in_arrs_1 = {}
for var in variables:
    listhere = []
    if var != 'id':
        for i in np.arange(12):
            listhere.append(list_of_var_in_arrs[i][var])
        var_in_arrs_1[var] =  np.concatenate(listhere,axis=0)

/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_01_Jan_release_MLS_ON
id
unbeachCount
time
3.982339859008789
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_02_Feb_release_MLS_ON
id
unbeachCount
time
3.7248852252960205
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_03_Mar_release_MLS_ON
id
unbeachCount
time
3.394073963165283
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_04_Apr_release_MLS_ON
id
unbeachCount
time
2.7813403606414795
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_05_May_release_MLS_ON
id
unbeachCount
time
2.998870849609375
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Forward(out_freq=5days)_Jan-Dec_MLS_ON/Month_06_Jun_release_MLS_ON
id
unbeachCount
time
2.

In [6]:
# Load traj files 2
out_freq    = traj_input[exp_name][2]['out_freq']
exp_path    = traj_input[exp_name][2]['exp_path']
dirs_2 = sorted(glob(master_path+exp_path+'*'))

# Files 1 are for 2005.01.01-2006.01.01,
# we applied a time selection to make sure no time step beyond is captured.
# Time here is in nanoseconds (1000000000 times of seconds) since 1970,1,1
time_max =  (dt.datetime(2006,1,1)  - dt.datetime(1970,1,1)).total_seconds()*1000000000
time_min =  (dt.datetime(2004,12,31)- dt.datetime(1970,1,1)).total_seconds()*1000000000

list_of_var_in_arrs = []
for i in dirs_2:
    print(i)
    files = sorted(glob(i+'/*.nc'))
    # Note that 'traj' is continous npts from 0, 
    # while 'trajectory' is particle id inherits from previous file
    t = time.time()
    #var_in_arrs = c1d.load_OP_traj_files_file_by_file(files,variables)
    var_in_arrs = c1d.load_OP_traj_files_file_by_file_v2(files,variables)
    print(time.time()-t)
    
    obs_mask = (var_in_arrs['time'][0,:]<time_max) & (var_in_arrs['time'][0,:]>time_min)
    for v in variables:
        if v != 'id':
            #print(v)
            var_in_arrs[v] = var_in_arrs[v][:,obs_mask]
            nan_arr = np.zeros((var_in_arrs[v].shape[0],int(365/out_freq)-var_in_arrs[v].shape[1]))/0
            var_in_arrs[v] = np.concatenate((nan_arr,var_in_arrs[v]),axis=1)
            #var_in_arrs_forw_1st_year = 
    list_of_var_in_arrs.append(var_in_arrs)
        
# Combine each monthly array into one array        
var_in_arrs_2 = {}
for var in variables:
    listhere = []
    if var != 'id':
        for i in np.arange(12):
            listhere.append(list_of_var_in_arrs[i][var])
        var_in_arrs_2[var] =  np.concatenate(listhere,axis=0)

/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_01_Jan_release_MLS_ON
id
unbeachCount
time
1.428316593170166
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_02_Feb_release_MLS_ON
id
unbeachCount
time
3.529160499572754
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_03_Mar_release_MLS_ON
id
unbeachCount
time
5.520434379577637
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_04_Apr_release_MLS_ON
id
unbeachCount
time
7.174632787704468
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_05_May_release_MLS_ON
id
unbeachCount
time
8.834099769592285
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Jan-Dec_MLS_ON/Month_06_Jun_release_MLS_ON
id
unbeachCount
tim

In [7]:
var_in_arrs_4={}
variables = {'id': 'trajectory', 'unbeachCount': 'unbeachCount'}
# Load traj files 4 # Note that No.4 means loop files
out_freq    = traj_input[exp_name][4]['out_freq']
exp_path    = traj_input[exp_name][4]['exp_path']

# Define the input files
files = sorted(glob(master_path+exp_path+'*.nc')) 
files = files[1:]
print(files[0])
print(files[-1])

t = time.time()
#var_in_arrs = c1d.load_OP_traj_files_file_by_file(files,variables)
var_in_arrs_4 = c1d.load_OP_traj_files_file_by_file_v2(files,variables)
print(time.time()-t)



/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Following_Loop_MLS_ON/Particles_001.nc
/g/data/jk72/yx9454/runs/parcels/output_after_Aug/CORE_Curtain_Backwards(out_freq=6hrs)_Following_Loop_MLS_ON/Particles_227.nc
id
unbeachCount
1466.9299602508545


# Load unbeachCount files

In [4]:
unbeachCount  = np.load("/g/data/jk72/yx9454/Euphotic_zone_research_trajectory_in_npz/MLS_ON/var_in_arrs_unbeachCount.npz")
#
var_in_arrs_0_UB = unbeachCount['var_in_arrs_0'][:,-1]
print('CP0')
var_in_arrs_1_UB = unbeachCount['var_in_arrs_1'][:,-1]
print('CP1')
var_in_arrs_2_UB = unbeachCount['var_in_arrs_2'][:,-1]
print('CP2')
var_in_arrs_4_UB = unbeachCount['var_in_arrs_4'][:,-1]
print('CP4')

CP0
CP1
CP2
CP4


### Load AABW indices

In [8]:
AABW_inds = np.load('/g/data/jk72/yx9454/PAR_from_iaf_cycle4/'+exp_name+'/AABW_inds.npz')['AABW_inds']


In [55]:
del unbeachCount
for unbeachCount in [var_in_arrs_0_UB,var_in_arrs_1_UB,var_in_arrs_2_UB,var_in_arrs_4_UB]:
    print('#--------------- chunk 1 start ---------------# ')
    print('Unbeached particle in all particles')
    num = np.nansum(unbeachCount > 0)
    print(num,' ',100*num/AABW_inds.shape[0],'%')
    print('Unbeached particle in AABW particles')
    if unbeachCount.shape[0] != 933504:
        # This is because we found var_in_arrs_4_UB length is less than AABW_inds, which is because
        # no 933504 particle is missing in var_in_arrs_4_UB
        num = np.nansum(unbeachCount[AABW_inds[:-1]] > 0)
    else:
        num = np.nansum(unbeachCount[AABW_inds] > 0)
    print(num,' ',100*num/AABW_inds.shape[0],'%')
    print('#--------------- chunk 1 finished ---------------# ')

print('#--------------- chunk 2 start ---------------# ')

int_unbeachCount = ((var_in_arrs_0_UB[:-1]) +
                    (var_in_arrs_1_UB[:-1]) +
                    (var_in_arrs_2_UB[:-1]) +
                    (var_in_arrs_4_UB) )

num = np.nansum(int_unbeachCount>0)
print(num,' ',100*num/AABW_inds.shape[0],'%')

#--------------- chunk 1 start ---------------# 
Unbeached particle in all particles
557072   59.67537364596188 %
Unbeached particle in AABW particles
71801   7.691557829425476 %
#--------------- chunk 1 finished ---------------# 
#--------------- chunk 1 start ---------------# 
Unbeached particle in all particles
114372   12.25190250925545 %
Unbeached particle in AABW particles
21249   2.2762623406005758 %
#--------------- chunk 1 finished ---------------# 
#--------------- chunk 1 start ---------------# 
Unbeached particle in all particles
142288   15.242355683532153 %
Unbeached particle in AABW particles
15997   1.7136509324009324 %
#--------------- chunk 1 finished ---------------# 
#--------------- chunk 1 start ---------------# 
Unbeached particle in all particles
574692   61.562885643768 %
Unbeached particle in AABW particles
58778   6.296491498697381 %
#--------------- chunk 1 finished ---------------# 
#--------------- chunk 2 start ---------------# 
763073   81.74287416015358

In [62]:
for thre in [0,7,10,100,496]:
    num = np.nansum(int_unbeachCount[AABW_inds[:-1]]>thre)
    print(num)
    print('Precentage: ',100*num/sum(AABW_inds),'%')

83334
Precentage:  88.53733943881942 %
43305
Precentage:  46.008945741210965 %
33000
Precentage:  35.06050593372502 %
8766
Precentage:  9.313345303485864 %
3132
Precentage:  3.327560744982629 %


In [61]:
for thre in [0,7,10,100,496]:
    num = np.nansum(int_unbeachCount>thre)
    print(num)
    print('Precentage: ',100*num/AABW_inds.shape[0],'%')

763073
Precentage:  81.74287416015358 %
366369
Precentage:  39.24664489921843 %
287312
Precentage:  30.777800630741808 %
85455
Precentage:  9.154218942821885 %
33907
Precentage:  3.6322286781845605 %
